## 信贷用户

In [1]:
import pandas as pd
import numpy as np
import networkx as nx
from igraph import *

In [2]:
#load Data
data = pd.read_csv('./xindai_2.csv')
data.shape,data.head()

((100495, 6),
              id          id_1          id_2          id_3  is_abnormal  \
 0  1.363927e+10           NaN  1.878687e+10  1.512023e+10            0   
 1  1.899508e+10  1.512198e+10  1.389532e+10  1.829570e+10            0   
 2  1.391930e+10  1.829326e+10  1.399326e+10  1.391971e+10            0   
 3  1.879515e+10           NaN  1.340952e+10  1.779551e+10            0   
 4  1.518083e+10  1.820851e+10  1.398416e+10  1.360850e+10            1   
 
    overdue  
 0      NaN  
 1      NaN  
 2      NaN  
 3      NaN  
 4      NaN  )

In [3]:
#预处理
# data.dropna(subset=['id'],inplace=True)
data['overdue'].fillna(0,inplace=True)
data.drop_duplicates(inplace=True)
data.loc[data[data.overdue>0].index,'is_abnormal']=2
data['is_abnormal'] = data['is_abnormal'].astype(int)
data.head()

,id,id_1,id_2,id_3,is_abnormal,overdue
0,1.363927e+10,NaN,1.878687e+10,1.512023e+10,0,0.0
1,1.899508e+10,1.512198e+10,1.389532e+10,1.829570e+10,0,0.0
2,1.391930e+10,1.829326e+10,1.399326e+10,1.391971e+10,0,0.0
3,1.879515e+10,NaN,1.340952e+10,1.779551e+10,0,0.0
4,1.518083e+10,1.820851e+10,1.398416e+10,1.360850e+10,1,0.0


### 统计

In [4]:
loan_time = data.groupby(by=['id']).count()['overdue']
loan_time =loan_time.sort_values(ascending=False)
loan_time =loan_time.reset_index().astype(int)
loan_time['label']=0
loan_time =loan_time.rename(columns={'overdue':'times'})
loan_time

,id,times,label
0,18085793563,3,0
1,13086277591,3,0
2,18335620462,3,0
3,13529588430,3,0
4,18685065956,3,0
5,13765325712,3,0
6,15117809689,3,0
7,15902641036,3,0
8,18286064983,3,0
9,13638027412,2,0


In [5]:
fm_time =data.groupby(by=['id_1']).count()['overdue'].sort_values(ascending=False).reset_index().astype(int)
fm_time['label']=3
fm_time =fm_time.rename(columns={'id_1':'id','overdue':'times'})
fm_time

,id,times,label
0,1008,137,3
1,1016,3,3
2,15812120871,2,3
3,15121845372,2,3
4,18209358868,2,3
5,15033535452,2,3
6,18685834347,2,3
7,15148984807,2,3
8,15693614947,2,3
9,18325368108,2,3


In [6]:
qs_time = data.groupby(by=['id_2']).count()['overdue'].sort_values(ascending=False).reset_index().astype(int)
qs_time['label']=2
qs_time = qs_time.rename(columns={'id_2':'id','overdue':'times'})
qs_time

,id,times,label
0,18183910801,4,2
1,13984580905,3,2
2,18683491012,3,2
3,13049519499,3,2
4,13111856527,3,2
5,15186441679,3,2
6,15379707977,3,2
7,15399370709,3,2
8,15519353899,3,2
9,13830901821,3,2


In [7]:
fr_time =data.groupby(by=['id_3']).count()['overdue'].sort_values(ascending=False).reset_index().astype(int)
fr_time['label']=1
fr_time = fr_time.rename(columns={'id_3':'id','overdue':'times'})
fr_time

,id,times,label
0,1008,39,1
1,13000001008,14,1
2,13982228130,10,1
3,18380210737,9,1
4,13258281890,9,1
5,15520743904,7,1
6,18628286386,7,1
7,13666264003,7,1
8,1009,7,1
9,18583904387,7,1


In [8]:
#merge
ab_id = loan_time[loan_time.times>2].append(fm_time[fm_time.times>1]).append(qs_time[qs_time.times>2])
ab_id = ab_id.append(fr_time[fr_time.times>2])

In [9]:
usr_df =pd.DataFrame()
usr_df['times'] = ab_id.groupby(by=['id'])['times'].sum()
usr_df['role'] = ab_id.groupby(by=['id'])['label'].apply(list)
usr_df

,times,role
id,,
1008,176,"[3, 1]"
1009,7,[1]
1016,3,[3]
13000001008,14,[1]
13007897663,4,[1]
13049519499,3,[2]
13056663334,2,[3]
13078726725,3,[1]
13086277591,3,[0]


In [10]:
usr_all = loan_time.append(fm_time).append(qs_time).append(fr_time)
usr_all_df = pd.DataFrame()
usr_all_df['times']=usr_all.groupby(by=['id'])['times'].sum()
usr_all_df['role'] = usr_all.groupby(by=['id'])['label'].apply(list)
usr_all_df

,times,role
id,,
-9223372036854775808,4,"[2, 2, 2, 2]"
1008,178,"[3, 2, 1]"
1009,9,"[2, 1]"
1014,2,[1]
1016,4,"[3, 2]"
1017,1,[1]
1021,1,[3]
1023,1,[2]
1026,1,[1]


In [23]:
usr_all_df.to_csv('./usr_all_time.csv',sep='\001')
usr_all_df[usr_all_df.times>2].to_csv('./usr_all_ab.csv',sep='\001')

In [21]:
usr_all_df[usr_all_df.times>2]['times'].value_counts()

3      803
4      106
5       29
6        8
7        5
9        3
178      1
15       1
10       1
Name: times, dtype: int64

### 建网络
节点属性   
- id,id1,3 #父母
- id,id2,2 #亲属
- id,id3,1 #朋友，同事
- id1,id2,2 #亲属
- id3,id2,0.5  #可能认识
- id3,id1,0.5  #可能认识

In [4]:
e1= data.dropna(subset=['id_1'])[['id','id_1']]
e1['weight']=3
e1.rename(columns={'id':'src','id_1':'dsc'},inplace=True)
e2 = data.dropna(subset=['id_2'])[['id','id_2']]
e2['weight']=2
e2.rename(columns={'id':'src','id_2':'dsc'},inplace=True)
e3 = data.dropna(subset=['id_3'])[['id','id_3']]
e3['weight']=1
e3.rename(columns={'id':'src','id_3':'dsc'},inplace=True)
e3.head()

,src,dsc,weight
0,1.363927e+10,1.512023e+10,1
1,1.899508e+10,1.829570e+10,1
2,1.391930e+10,1.391971e+10,1
3,1.879515e+10,1.779551e+10,1
4,1.518083e+10,1.360850e+10,1


In [5]:
tmp = e1.append(e2)
tmp = tmp.append(e3)
tmp = tmp.groupby(by=['src','dsc'])['weight'].sum()
tmp = tmp.reset_index()
# tmp = tmp.astype(np.int)
tmp.shape,tmp.head()

((225742, 3),             src           dsc  weight
 0  1.300000e+10  1.390000e+10       1
 1  1.300010e+10  1.398687e+10       1
 2  1.300100e+10  1.587742e+10       1
 3  1.300100e+10  1.589105e+10       2
 4  1.300141e+10  1.366315e+10       2)

In [6]:
e4 = data.dropna(subset=['id_1','id_2'])[['id_1','id_2']]
e4['weight']=2
e4.rename(columns={'id_1':'src','id_2':'dsc'},inplace=True)
e5 = data.dropna(subset=['id_2','id_3'])[['id_2','id_3']]
e5['weight']=0.5
e5.rename(columns={'id_2':'src','id_3':'dsc'},inplace=True)
e6 = data.dropna(subset=['id_1','id_3'])[['id_1','id_3']]
e6['weight']=0.5
e6.rename(columns={'id_1':'src','id_3':'dsc'},inplace=True)
e6.head()

,src,dsc,weight
1,1.512198e+10,1.829570e+10,0.5
2,1.829326e+10,1.391971e+10,0.5
4,1.820851e+10,1.360850e+10,0.5
5,1.534690e+10,1.534690e+10,0.5
8,1.310760e+10,1.389752e+10,0.5


In [7]:
tmp1 = e4.append(e5)
tmp1 = tmp1.append(e6)
tmp1 = tmp1.groupby(by=['src','dsc'])['weight'].sum()
tmp1 = tmp1.reset_index()
# tmp1 =tmp1.astype(np.int)
tmp1.shape,tmp1.head()

((159604, 3),       src           dsc  weight
 0  1008.0  1.359596e+09     0.5
 1  1008.0  1.828561e+09     2.0
 2  1008.0  1.308699e+10     2.0
 3  1008.0  1.309680e+10     0.5
 4  1008.0  1.311860e+10     0.5)

In [8]:
tmp2 = tmp.append(tmp1)
tmp2 = tmp2.groupby(by=['src','dsc'])['weight'].sum()
tmp2 = tmp2.reset_index()
tmp2.shape

(384327, 3)

## networks 社团划分

In [9]:
#network 1
G1 = nx.from_pandas_dataframe(tmp,'src','dsc',['weight'])
print G1.number_of_nodes()
print G1.number_of_edges()
# #network 2
# G2 = nx.from_pandas_dataframe(tmp2,'src','dsc',['weight'])
# print G2.number_of_nodes()
# print G2.number_of_edges()

314260
224654


In [16]:
loop_nodes=pd.DataFrame()
loop_nodes['id'] = G1.nodes_with_selfloops()
loop_nodes = loop_nodes.merge(data[['id','is_abnormal']],on='id',how='left')
loop_nodes.drop_duplicates(inplace=True)
loop_nodes['is_abnormal'].value_counts()

0    1060
1     417
2     138
Name: is_abnormal, dtype: int64

In [19]:
nx.average_degree_connectivity(G1)

{1: 2.4170467076612527,
 2: 1.1726165106332946,
 3: 1.4186963504458106,
 4: 1.6509757383966244,
 5: 1.73416149068323,
 6: 1.6435185185185186,
 7: 1.8181818181818181,
 8: 1.875,
 9: 1.9444444444444444,
 10: 2.0,
 15: 2.466666666666667,
 178: 2.938202247191011}

In [11]:
#nodes attribute
# data[id] = data['id'].astype(int)
# tmp3 = data.groupby(by=['id'])['is_abnormal'].apply(lambda a:list(a)[-1])
# for id in tmp3.index:
#     try:
#         G1.node[id]['label']=tmp3.at[id]
#         G2.node[id]['label']=tmp3.at[id]
#     except:
#         G1.add_node(id,label=tmp3.at[id])
#         G2.add_node(id,label=tmp3.at[id])
# G1.nodes(data=True)

In [12]:
# nx.get_node_attributes(G1,'label')
# print nx.get_node_attributes(G2,'label')

In [13]:
# communities = list(nx.k_clique_communities(G1,4))
# communities

# sub_graph = nx.subgraph(G1,['13985776740', '15286214503', '18285456348', '18932045810'])
# sub_graph.number_of_edges()

# data[data.id=='15286214503']

# data[data.id=='13985776740']

# com2 = list(nx.k_clique_communities(G2,5))
# com2

# data[data.id=='13892193002']

# data[data.id=='13571140739']

# sub_g2 =nx.subgraph(G2,set(['13484654728','13571140739','13571158670','13892193002','15319574098','13984979849',
#                             '14708602047','14785677001','15186637987','15348682422']))
# sub_g2.edges(data=True)

# import matplotlib.pyplot as plt
# nx.draw_networkx(sub_graph)

# plt.show()

## igraph
节点必须为integer类型，需要对电话进行枚举重新编码

In [14]:
#重新编码
id=data['id'].dropna().tolist()
id_1= data['id_1'].dropna().tolist()
id_2 =data['id_2'].dropna().tolist()
id_3=data['id_3'].dropna().tolist()

In [15]:
ids =set(id+id_1+id_2+id_3)
id_df=pd.DataFrame(data=list(ids),columns=['ids'])
id_df.head()

,ids
0,1.593469e+10
1,1.370856e+10
2,1.828524e+10
3,1.346852e+10
4,1.874854e+10


In [16]:
id_dict = id_df.reset_index().set_index('ids').to_dict()['index']
id_dict
tmp['src']=tmp['src'].map(id_dict)
tmp['dsc']=tmp['dsc'].map(id_dict)
tmp.head()

,src,dsc,weight
0,166921,45972,1
1,226330,262024,1
2,138822,240090,1
3,138822,243542,2
4,206449,123165,2


In [17]:
#构建网络
edges=zip(tmp['src'],tmp['dsc'],tmp['weight'])
g = Graph.TupleList(edges,edge_attrs=['weight'])
g.summary()

'IGRAPH UNW- 314260 225742 -- \n+ attr: name (v), weight (e)'

In [18]:
#社团划分,合并相同节点间的边和自环
uG = g.simplify(combine_edges =sum)
fg = uG.community_fastgreedy(weights = 'weight')
fg

In [29]:
#缓存
import pickle
f = open('communities.txt', 'wb')
pickle.dump(communities, f)
f.close()

In [20]:
#社团大小大于4的社团
communities = [l for l in fg.as_clustering()  if len(l)>5 ]
len(communities)

2756

In [26]:
nodes = []
for com in communities:
    nodes =nodes+com
len(nodes)

19791

In [ ]:
sub_g = g.subgraph(nodes)
plot(sub_g,'sub_g.pdf',layout=sub_g.layout('kk'),mark_groups=True)

In [75]:
#节点属性
tmp3 = data.groupby(by=['id'])['is_abnormal'].apply(lambda a:list(a)[-1])
tmp3.index = tmp3.reset_index()['id'].map(id_dict)
tmp3

id
280328    1
166921    1
226330    1
138822    1
206449    0
77414     0
115275    0
6485      1
26805     1
75560     1
18675     0
23920     0
33871     0
132529    0
285452    0
300373    0
308429    1
7071      1
7447      1
9951      0
13170     0
19129     1
23878     2
32378     1
39878     0
42366     0
53557     0
55016     0
75615     1
76737     1
         ..
297980    0
667       0
7074      0
10397     1
11013     0
208010    0
69954     0
74855     0
76093     0
76263     0
162631    0
91017     1
104167    0
113854    0
125883    1
226509    0
160649    0
271926    0
208977    0
209770    1
255050    0
273026    0
57228     1
123189    0
34997     0
60557     0
99330     0
163299    0
178619    1
308522    0
Name: is_abnormal, Length: 99094, dtype: int64

In [88]:
#usr feature
usr_f =pd.DataFrame()
usr_f['loan_time'] = data.groupby(by=['id']).count()['overdue']
usr_f['loan_id1'] = data.groupby(by=['id'])['id_1'].apply(lambda a:len(set(a)))
usr_f.shape,usr_f.head()
data = data.merge(usr_f.reset_index(),on='id',how='left')
data

((99094, 2),                        loan_time  loan_id1
 id                                        
 1.55871559001376E+021          1         1
 12345662244                    1         1
 13000001008                    1         1
 13000099806                    1         1
 13001002738                    1         1)

In [90]:
#统计
data[data.is_abnormal=='1'].shape  #29455
data[data.overdue.notnull()]['overdue'].value_counts()

0    90893
1     3654
4     1650
2     1361
3     1077
5      789
7      238
6       64
Name: overdue, dtype: int64

In [155]:
edge3 = data[['id','id_3','is_abnormal']]
edge3 = edge3.groupby(by=['id','id_3'])['is_abnormal'].apply(lambda a:list(a)[-1])
edge3 =edge3.reset_index()
edge3

,id,id_3,is_abnormal
0,1.55871559001376E+021,18313887294,0
1,13000001008,13900001008,1
2,13000099806,13986868817,1
3,13001002738,15877415138,1
4,13001411187,15127561315,0
5,13001424290,13903353168,0
6,13001487652,18134371238,0
7,13001863449,15231028714,1
8,13001945455,15650929428,1
9,13002373897,15683113259,0
